In [1]:
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from models import AlexNet
from keras import backend as K
K.set_image_data_format('channels_last')

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_dataset = BatchGenerator(box_output_format=['class_id'])
validation_dataset = BatchGenerator(box_output_format=['class_id'])

train_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression_train_split_47950-70-10-20.csv', 
                        images_dir='../../ssd_keras/dataset/',
                        input_format=['image_name', 'class_id'])

validation_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression_val_split_47950-70-10-20.csv', 
                             images_dir='../../ssd_keras/dataset/',
                             input_format=['image_name', 'class_id'])

In [14]:
img_height, img_width, img_depth = (227,227,3)

epochs = 1

train_batch_size = 64
shuffle = True
ssd_train = False

validation_batch_size = 32


In [15]:
train_generator = train_dataset.generate(batch_size=train_batch_size,
                                         shuffle=shuffle,
                                         ssd_train=ssd_train,
                                         returns={'processed_labels'},
                                         resize=(img_height, img_width))

validation_generator = validation_dataset.generate(batch_size=validation_batch_size,
                                                   shuffle=shuffle,
                                                   ssd_train=ssd_train,
                                                   returns={'processed_labels'},
                                                   resize=(img_height, img_width))

print("Number of images in the dataset:", train_dataset.get_n_samples())
print("Number of images in the dataset:", validation_dataset.get_n_samples())

Number of images in the dataset: 33565
Number of images in the dataset: 4795


In [16]:
steps_per_epoch = train_dataset.get_n_samples()/train_batch_size
validation_steps = validation_dataset.get_n_samples()/validation_batch_size

In [6]:
alexnet = AlexNet(n_classes=1)

/home/nicoli/github/deep-learning-learning/redes-tcc/models.py:88: UserWarning: Considering a regression task with output function being 'relu'
  warnings.warn('Considering a regression task with output function being \'relu\'')


In [7]:
alexnet.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 57, 57, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 57, 57, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 96)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 28, 28, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 256)       1024      
_________________________________________________________________
convpool_1 (MaxPooling2D)    (None, 13, 13, 256)       0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 13, 13, 384)       885120    
__________

In [8]:
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00001, amsgrad=True)

In [17]:
checkpoint = ModelCheckpoint(filepath='callbacks/alexnet/age/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=False,
                           period=5)
callbacks = [checkpoint]

In [18]:
alexnet.model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse', 'mae'])

In [19]:
alexnet.model.fit_generator(train_generator, epochs=epochs, 
                             steps_per_epoch=steps_per_epoch, 
                             validation_data=validation_generator,
                             validation_steps=validation_steps,
                             callbacks=callbacks)

Epoch 1/1
  7/524 [..............................] - ETA: 6:53:52 - loss: 63195304525488528.0000 - mean_squared_error: 63195304525488528.0000 - mean_absolute_error: 66920208.0188

KeyboardInterrupt: 